In [1]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./botswana.tsv", sep="\t")
# df

In [3]:
df.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [4]:
df.loc[np.alltrue(df.notnull(), axis=1),:].shape

(1834, 15)

In [5]:
df = pd.read_csv("./botswana.tsv", sep="\t")

df['nevermarr'] = 0
df.loc[df['evermarr'] == 0, 'nevermarr'] = 1
df.drop('evermarr', axis=1, inplace=True)

df.loc[(df['nevermarr'] == 1)&(df.agefm.isnull()), 'agefm'] = 0
df.loc[(df['nevermarr'] == 1)&(df['heduc'].isnull()), 'heduc'] = -1
# df.loc[
#     (df['idlnchld'].isnull())&(df['nevermarr'] == 1), 
#     'idlnchld'
# ] = -1
# df.loc[
#     (df['usemeth'].isnull())&(df['nevermarr'] == 1), 
#     'usemeth'
# ] = -1
print "#3 = ", sum(df.heduc.isnull())

#4.
df['idlnchld_noans'] = df['idlnchld'].isnull().astype(int)
df.idlnchld.fillna(-1, inplace=True)

df['heduc_noans'] = df['heduc'].isnull().astype(int)
df.heduc.fillna(-1, inplace=True)

df['usemeth_noans'] = df['usemeth'].isnull().astype(int)
df.usemeth.fillna(-1, inplace=True)

#3 =  123


In [6]:
df = df.loc[
    np.alltrue(
        df.loc[
            :, 
            ['knowmeth', 'electric', 'radio', 'tv', 'bicycle']
        ].notnull(), 
        axis=1
    ),
    :
]

# 4.
df.shape[0] * df.shape[1]

78264

In [7]:
# 5. 6.
m1 = smf.ols(
    'ceb~age+educ+religion+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans',
    data=df
)
fitted1 = m1.fit()
fitted1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sat, 12 Jan 2019   Prob (F-statistic):               0.00
Time:                        15:13:13   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans             0.6539      0.153      4.286      0.000       0.355       0.953
heduc_noans               -0.8173      0.142     -5.756      0.000      -1.096      -0.539
usemeth_noans              0.7652      0.196      3.910      0.000       0.382       1.149
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         360.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
#7.
fitted1 = m1.fit(cov_type='HC1')
sms.het_breuschpagan(fitted1.resid, fitted1.model.exog)[1]

1.1452927633437192e-225

In [9]:
# 8.
m2 = smf.ols(
#     'ceb~age+educ+religion+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans',
    'ceb~age+educ+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans',
    data=df
)

fitted2 = m2.fit(cov_type='HC1')

# sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1]
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(fitted2)

F=0.919236, p=0.467231, k1=5.000000


/Users/roman/py2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1896: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


In [11]:
m3 = smf.ols(
#   'ceb~age+educ+religion+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans',
    'ceb~age+educ+idlnchld+knowmeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans',
    data=df
)

# fitted = m.fit()

# m = smf.ols(
#     'ceb~age+educ+idlnchld+knowmeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans',
#     data=df
# )

fitted3 = m3.fit(cov_type='HC1')
# fitted.summary()

f,p,k1 = m2.fit().compare_f_test(fitted3)
#9
p

/Users/roman/py2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1896: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


3.1552009480386492e-40

In [14]:
fitted2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sat, 12 Jan 2019   Prob (F-statistic):               0.00
Time:                        15:42:14   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8280      0.188     -4.412      0.000      -1.196      -0.460
usemeth_noans      0.7732      0.212      3.639      0.000       0.357       1.190
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [28]:
0.6565 + 0.0770*(-1)

0.5795